# Moonshot 5 Hackathon - Drought in the Netherlands
Using RA2CE to assess the capacity of the road network between Rotterdam and the Ruhr area.

In [39]:
from ra2ce.ra2ce_handler import Ra2ceHandler # import the ra2cehandler to run ra2ce analyses
from pathlib import Path
import networkx as nx
import pandas as pd
from ra2ce.common.io.readers.graph_pickle_reader import GraphPickleReader
from ra2ce.graph.exporters.multi_graph_network_exporter import MultiGraphNetworkExporter

_network_ini_name = "network.ini" # set the name for the network.ini settings file

folder_dir = Path(r'C:\Users\groen_fe\OneDrive - Stichting Deltares\1_Projects\Moonshot_5')

network_ini = folder_dir / _network_ini_name
assert network_ini.is_file() # check whether the network.ini file exists

Download the road network from OSM.

In [13]:
race = Ra2ceHandler(network=network_ini, analysis=None)
race.configure()

2023-09-15 03:46:56 PM - [ra2ce_logging.py:41] - root - INFO - RA2CE logger initialized.
2023-09-15 03:46:56 PM - [ra2ce_logging.py:41] - root - INFO - RA2CE logger initialized.
2023-09-15 03:46:56 PM - [ra2ce_logging.py:41] - root - INFO - RA2CE logger initialized.
2023-09-15 03:46:56 PM - [ra2ce_logging.py:41] - root - INFO - RA2CE logger initialized.
2023-09-15 03:46:56 PM - [ra2ce_logging.py:41] - root - INFO - RA2CE logger initialized.
2023-09-15 03:46:56 PM - [ra2ce_logging.py:41] - root - INFO - RA2CE logger initialized.
2023-09-15 03:46:56 PM - [ra2ce_logging.py:41] - root - INFO - RA2CE logger initialized.
2023-09-15 03:46:56 PM - [ra2ce_logging.py:41] - root - INFO - RA2CE logger initialized.
2023-09-15 03:46:56 PM - [network_config_wrapper.py:106] - root - INFO - Existing graph/network found: C:\Users\groen_fe\OneDrive - Stichting Deltares\1_Projects\Moonshot_5\static\output_graph\base_graph.p.
2023-09-15 03:46:56 PM - [network_config_wrapper.py:106] - root - INFO - Existing

We can inspect the created graph

In [16]:
od_graph = folder_dir / "static" / "output_graph" / "origins_destinations_graph.p"
graph = GraphPickleReader().read(od_graph)

We pick the lowest number of lanes per road segment and assign 1 lane to the road segments without data.

In [37]:
for e in graph.edges(data=True, keys=True):
    if "lanes" not in e[-1]:
        nr_lanes = 1
    else:
        nr_lanes = e[-1]["lanes"]
    if isinstance(nr_lanes, list):
        nr_lanes = min([int(x) for x in nr_lanes])
    if isinstance(nr_lanes, str):
        nr_lanes = int(nr_lanes)
    graph[e[0]][e[1]][e[2]]["lanes_new"] = nr_lanes


We translate the number of lanes to capacity.

In [45]:
capacity = folder_dir / "road_capacity.csv"
capacity = pd.read_csv(capacity)
ffs_list = capacity["FFS (km/h)"].to_list()

for e in graph.edges(data=True, keys=True):
    ffs = min(ffs_list, key=lambda x: abs(x - e[-1]["lanes_new"]))
    graph[e[0]][e[1]][e[2]]["capacity"] = capacity.loc[capacity["FFS (km/h)"] == ffs, "E (pc/h/ln)"].values[0] * e[-1]["lanes_new"]

In [46]:
exporter = MultiGraphNetworkExporter(basename="updated_network", export_types=["shp"])
exporter.export_to_shp(output_dir=folder_dir / "static" / "output_graph", export_data=graph)

2023-09-15 04:53:44 PM - [networks_utils.py:1210] - root - INFO - Saving nodes as shapefile: C:\Users\groen_fe\OneDrive - Stichting Deltares\1_Projects\Moonshot_5\static\output_graph\updated_network_nodes.gpkg
2023-09-15 04:53:44 PM - [networks_utils.py:1210] - root - INFO - Saving nodes as shapefile: C:\Users\groen_fe\OneDrive - Stichting Deltares\1_Projects\Moonshot_5\static\output_graph\updated_network_nodes.gpkg
2023-09-15 04:53:44 PM - [networks_utils.py:1210] - root - INFO - Saving nodes as shapefile: C:\Users\groen_fe\OneDrive - Stichting Deltares\1_Projects\Moonshot_5\static\output_graph\updated_network_nodes.gpkg
2023-09-15 04:53:44 PM - [networks_utils.py:1210] - root - INFO - Saving nodes as shapefile: C:\Users\groen_fe\OneDrive - Stichting Deltares\1_Projects\Moonshot_5\static\output_graph\updated_network_nodes.gpkg
2023-09-15 04:53:44 PM - [networks_utils.py:1210] - root - INFO - Saving nodes as shapefile: C:\Users\groen_fe\OneDrive - Stichting Deltares\1_Projects\Moonshot